__собираем данные__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/flat/avito' 
cian_raw_data_path = 'data/raw/flat/cian'

result_file_path = 'data/data_flat.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/flat/avito/avito_2022-08-26_17-29_raw.xlsx',
 'data/raw/flat/avito/avito_2022-08-29_13-16_raw.xlsx',
 'data/raw/flat/avito/avito_2022-08-31_17-42_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-01_14-07_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-02_13-04_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-05_14-02_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-06_13-09_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-07_12-39_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-08_13-36_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-12_13-44_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-13_13-55_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-15_10-27_raw.xlsx']

In [7]:
from lib.avito import AvitoDataCleanerRealtyFlat

data_avito = pd.concat([
        pd
        .read_excel(f)
        .dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

data_avito = AvitoDataCleanerRealtyFlat().transform( data_avito )

print(len(data_avito))

18459


In [8]:
assert len(data_avito)>0

In [9]:
data_avito.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18459 entries, 0 to 18458
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   avito_id       18459 non-null  int64         
 1   title          18459 non-null  object        
 2   price          18459 non-null  int64         
 3   adr            18459 non-null  object        
 4   description    18459 non-null  object        
 5   obj_name       18459 non-null  object        
 6   ts             18459 non-null  datetime64[ns]
 7   avito_page     18459 non-null  int64         
 8   nrooms         18459 non-null  int64         
 9   floor          18459 non-null  int64         
 10  nfloors        18459 non-null  int64         
 11  area           18459 non-null  float64       
 12  is_studio      18459 non-null  bool          
 13  is_apartment   18459 non-null  bool          
 14  is_part        18459 non-null  bool          
 15  is_auction     1845

In [10]:
# data_avito.query('nrooms==0')
# data_avito[ data_avito['title'].str.len()<1]

## собираем данные циан.ру

In [11]:
import re
from os import listdir

raw_data_files = sorted([
        cian_raw_data_path+'/'+f 
        for f in listdir(cian_raw_data_path) 
        if re.match(r'.+\.pkl$',f)
        #if re.match(r'.+\.xlsx$',f)
    
    ])
raw_data_files

['data/raw/flat/cian/cian_2022-09-08_14-25_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-12_13-59_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-13_13-59_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-15_10-30_raw.pkl']

In [12]:
data_cian = pd.concat([
        # pd.read_excel(f)
        pd.read_pickle(f)
         .dropna()
         .drop_duplicates(['LinkArea','Description',])
         .rename(columns={'page':'cian_page'})
         .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

In [13]:
data_cian.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6022 entries, 0 to 6021
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   OfferTitle     6022 non-null   object        
 1   OfferSubtitle  6022 non-null   object        
 2   Deadline       6022 non-null   object        
 3   MainPrice      6022 non-null   object        
 4   PriceInfo      6022 non-null   object        
 5   GeoLabel       6022 non-null   object        
 6   TimeLabel      6022 non-null   object        
 7   LinkArea       6022 non-null   object        
 8   Description    6022 non-null   object        
 9   cian_page      6022 non-null   int64         
 10  ts             6022 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(9)
memory usage: 517.6+ KB


In [14]:
swap_title = data_cian['OfferSubtitle'].str.match(r'.*\d+ м², \d+/\d+ эт.*') 

data_cian = pd.concat([
        data_cian[ ~swap_title ],
        data_cian[ swap_title ]
            .rename(columns={'OfferTitle':'OfferSubtitle','OfferSubtitle':'OfferTitle'})
    ]).reset_index(drop=True)


In [15]:
# data_cian

In [16]:
from lib.cian import CianDataCleaner

data_cian = CianDataCleaner().transform( data_cian )

print(len(data_cian))

6022


In [17]:
assert len(data_cian)>0

In [18]:
data_cian.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6022 entries, 0 to 6021
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   title          6022 non-null   object        
 1   obj_name       6022 non-null   object        
 2   adr            6022 non-null   object        
 3   nrooms         6022 non-null   int64         
 4   floor          6022 non-null   int64         
 5   nfloors        6022 non-null   int64         
 6   area           6022 non-null   float64       
 7   is_studio      6022 non-null   bool          
 8   is_apartment   6022 non-null   bool          
 9   is_part        6022 non-null   bool          
 10  is_auction     6022 non-null   bool          
 11  is_openspace   6022 non-null   bool          
 12  is_roof        6022 non-null   bool          
 13  is_SNT         6022 non-null   bool          
 14  price          6022 non-null   int64         
 15  priceM         6022 n

In [19]:
# data_cian
# data_cian.query('nrooms==0')
# data_cian[ data_cian['title'].str.len()<1 ]
# data_cian[ data_cian['obj_name'].str.len()>0 ][['title','obj_name',]] # .to_csv('tmp/cian_title.csv',sep='\t')|

----

In [20]:
data = pd.concat([data_avito,data_cian]).reset_index(drop=True) 
print(len(data))
assert len(data)>0

24481


### обновляем таблицу адресов

In [ ]:
from lib.locator import AddressTransformer

data['adr'] = AddressTransformer().transform( data['adr'] )

places = {
#'bahchisaray': 'Бахчисарай',
#      'sudak': 'Судак',
 'sevastopol': 'Севастополь',
}

for k,v in places.items():
    print(k,v)
    f =  (data['place']==k) & (~data['adr'].str.match(v))
    data.loc[f,'adr'] = v +', '+ data.loc[f,'adr']

In [21]:
from lib.locator import LocationUpdater
# from lib.locator import AddressTransformerSev
from lib.locator import GeocoderYandex
from yandex_keys import keys

loc_file_path = 'data/location.pkl'# таблица адресов

loc = LocationUpdater(
        locator=GeocoderYandex(keys[0]),
        # address_transformer=AddressTransformerSev(),
    ).transform(
        adr=data['adr'],
        loc=pd.read_pickle(loc_file_path),
        show_pbar=True,
    )

[INFO    ] 2022-09-15 10:36:29 | LocationUpdater: 3616 addresses in location table
[INFO    ] 2022-09-15 10:36:29 | LocationUpdater: 3667 addresses total
[INFO    ] 2022-09-15 10:36:29 | LocationUpdater: 3616 addresses defined
[INFO    ] 2022-09-15 10:36:29 | LocationUpdater: 51 addresses undefined


  0%|          | 0/51 [00:00<?, ?it/s]

[INFO    ] 2022-09-15 10:36:46 | LocationUpdater: 51 new addresses found


In [22]:
# from lib.locator import LocationUpdater
# from lib.locator import AddressTransformerSev

# loc_file_path = 'data/location.pkl'# таблица адресов

# loc = LocationUpdater(
#         address_transformer=AddressTransformerSev(),
#     ).transform(
#         adr=data['adr'],
#         loc=pd.read_pickle(loc_file_path),
#         show_pbar=True,
#     )

In [23]:
loc.sample(3)

,latitude,longitude,adr,truncated
913,44.57,33.50,"улица Военных Строителей, стр. 6.2",NaN
2003,44.56,33.53,"проспект Генерала Острякова, 244 к4",False
544,44.50,33.60,"Балаклава, ул. Кирова, 33",NaN


In [24]:
assert len(loc)>0
loc.to_pickle(loc_file_path)

### дополняем данные геометкой

In [25]:
data = data.merge(loc[['adr','latitude','longitude',]],on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isnull() ] ) )

всего записей: 24481
записей без геометки: 0


In [26]:
assert len(data)>0
data.to_pickle(result_file_path)

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24481 entries, 0 to 24480
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   avito_id       18459 non-null  float64       
 1   title          24481 non-null  object        
 2   price          24481 non-null  int64         
 3   adr            24481 non-null  object        
 4   description    24481 non-null  object        
 5   obj_name       24481 non-null  object        
 6   ts             24481 non-null  datetime64[ns]
 7   avito_page     18459 non-null  float64       
 8   nrooms         24481 non-null  int64         
 9   floor          24481 non-null  int64         
 10  nfloors        24481 non-null  int64         
 11  area           24481 non-null  float64       
 12  is_studio      24481 non-null  bool          
 13  is_apartment   24481 non-null  bool          
 14  is_part        24481 non-null  bool          
 15  is_auction     2448

In [28]:
# for f in raw_data_files:
#     ts = dtm.strptime( re.sub(r'.*/avito_','',f), '%Y-%m-%d_%H-%M_raw.xlsx')
#     df = pd.read_excel(f)
#     df['ts'] = ts
#     df.to_excel(f,index=False)
#     print(ts)

----

In [29]:
# from tqdm.notebook import tqdm
# import pandas as pd
# tqdm.pandas()

In [30]:
# result_file_path = 'data/data_flat.pkl'
# data = pd.read_pickle(result_file_path)

In [31]:
# data[ data['longitude'].isnull() ][['title','adr','latitude','longitude']]

In [32]:
# from lib.locator import LocationUpdater
# from lib.locator import AddressTransformerSev

In [33]:
# adr = data[ data['longitude'].isnull() ].sample(10)[['adr']]
# adr['adr_'] = AddressTransformerSev().transform(adr['adr'])
# adr

In [34]:
# from geopy.geocoders import Nominatim

# user_agent='Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0'
# geolocator = Nominatim(user_agent=user_agent)

In [35]:
# adr['adr_']

In [36]:
# def coo(s,geolocator=geolocator):
#     loc = geolocator.geocode(s)
#     if not (loc is None):
#         return { 'latitude':loc.latitude,'longitude':loc.longitude}
#     return None
    
# adr['adr_'].progress_apply( coo)

In [37]:
# loc_file_path = 'data/location.pkl'# таблица адресов

In [38]:
# loc = LocationUpdater(
#         address_transformer=AddressTransformerSev(),
#     ).transform(
#         adr=adr['adr'],
#         loc=pd.read_pickle(loc_file_path),
#         show_pbar=True,
#     )

# loc

In [39]:
# loc=pd.read_pickle(loc_file_path)
# loc[loc['longitude'].isnull()]